# Wires från dgn till gdb
## fas och topp för sig
## littera som attribut

In [1]:
import arcpy, os, glob
from pathlib import Path
import pandas as pd

prj_dir = r"C:\SVK_2023\pythonkörningar\2020-12-20"
dgn_dir = r"U:\Projekt\Analys_2023\DGN"
gdb  = os.path.join(prj_dir, "RBX_2023_12_20.gdb", "Wires")

LGs_info_path = r"U:\Projekt\Data_2023\styrfiler\RBX_kantträd\2023-12-20.txt"
cvd_LEDNINGSGATA_path = r"U:\Projekt\Data\Geodataunderlag_SVK\cvd_LEDNINGSGATA.txt"

arcpy.env.overwriteOutput = True

def wires_to_gdb(row):
    LG = row["LG"]
    line = row["line"]
    littera = row["Littera"]
    LG_code = cvd_LEDNINGSGATA[LG]
    
    print(f"{LG}_{line}, {littera}")
    
    dgn = os.path.join(dgn_dir, f"{LG}_{line}.dgn")
    arcpy.env.workspace = dgn_dir
    
    # faslinor
    fc_fas = f"{LG}_{line}_fas"
    # Create a value table that will hold the input feature classes for Merge
    vTab_fas = arcpy.ValueTable()
    print("vTab created")
    layer_fas = "Layer_fas"
    arcpy.Delete_management(layer_fas)
    arcpy.MakeFeatureLayer_management(dgn + '/Polyline', layer_fas, "\"Level\" = 1")
    print("MakeFeatureLayer complete")
    vTab_fas.addRow(layer_fas)
    print("Row added to vTab_fas")
        
    # topplinor
    fc_topp = f"{LG}_{line}_topp"
    # Create a value table that will hold the input feature classes for Merge
    vTab_topp = arcpy.ValueTable()
    print("vTab created")
    layer_topp = "Layer_topp"
    arcpy.Delete_management(layer_topp)
    arcpy.MakeFeatureLayer_management(dgn + '/Polyline', layer_topp, "\"Level\" = 2")
    print("MakeFeatureLayer complete")
    vTab_topp.addRow(layer_topp)
    print("Row added to vTab_topp")

    
    # Merge the CAD features into one feature class
    arcpy.env.workspace = gdb
    
    # faslinor
    arcpy.Merge_management(vTab_fas, fc_fas)
    # Add and populate littera field
    arcpy.management.AddField(fc_fas, "Littera", "TEXT")
    arcpy.management.CalculateField(fc_fas, "Littera", "".join(("'",littera,"'")))
    arcpy.management.AddField(fc_fas, "LG", "LONG")
    arcpy.management.CalculateField(fc_fas, "LG", int(LG_code))
    arcpy.management.AddField(fc_fas, "lintyp", "TEXT")
    arcpy.management.CalculateField(fc_fas, "lintyp", "'ledare'")
    # domains
    arcpy.management.AssignDomainToField(fc_fas, "LG", "cvd_LEDNINGSGATA")
    arcpy.management.AssignDomainToField(fc_fas, "Littera", "cvd_LITTERA_LEDNING")
    
    
    # topplinor
    arcpy.Merge_management(vTab_topp, fc_topp)
    # Add and populate littera field
    arcpy.management.AddField(fc_topp, "Littera", "TEXT")
    arcpy.management.CalculateField(fc_topp, "Littera", "".join(("'",littera,"'")))
    arcpy.management.AddField(fc_topp, "LG", "LONG")
    arcpy.management.CalculateField(fc_topp, "LG", int(LG_code))
    arcpy.management.AddField(fc_topp, "lintyp", "TEXT")
    arcpy.management.CalculateField(fc_topp, "lintyp", "'topplina'")
    # domains
    arcpy.management.AssignDomainToField(fc_fas, "LG", "cvd_LEDNINGSGATA")
    arcpy.management.AssignDomainToField(fc_fas, "Littera", "cvd_LITTERA_LEDNING")
    

LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
df_cvd_LEDNINGSGATA = pd.read_csv(cvd_LEDNINGSGATA_path, sep="\t", header=0)
cvd_LEDNINGSGATA = {df_cvd_LEDNINGSGATA.LG[i]: df_cvd_LEDNINGSGATA.Code[i] for i in range(len(df_cvd_LEDNINGSGATA))}


LGs_info.apply(wires_to_gdb, axis=1)

LG012_3, UL1 S1-3
vTab created
MakeFeatureLayer complete
Row added to vTab_fas
vTab created
MakeFeatureLayer complete
Row added to vTab_topp
LG056_1, RL8 S7
vTab created
MakeFeatureLayer complete
Row added to vTab_fas
vTab created
MakeFeatureLayer complete
Row added to vTab_topp


0    None
1    None
dtype: object